# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [X] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Marco Paredes (A15951023)
- Alan Hang (A16409702)
- Zehong Li (A15852954)
- Danyal Iqbal (A16687740)

<a id='research_question'></a>
# Research Question

*Are covid cases a good predictor of the price of cryptocurrency?*

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*We hypothesize that there is a positive correlation between covid cases and the price of cryptocurrencies. The reason for this being that there has been historical evidence of a continual rise of Bitcoin and Etheruem prices since the start of covid in the US. Investors may have found the decentralized nature of cryptocurrency more attractive when the economy of coutnries worldwide became shaky during the global pandemic.*

# Dataset(s)

- Dataset Name: Daily Ethereum(ETH) data
- Link to the dataset: https://raw.githubusercontent.com/alhang/csv-files/master/gemini_ETHUSD_day.csv
- Number of observations: 2101
- Description: Daily Ethereum(ETH) data will be used to see daily Bitcoin trends between 2019-2021. The data will indicate changes of the respective stock prices through the year


- Dataset Name: Daily Bitcoin(BTC) data
- Link to the dataset: https://raw.githubusercontent.com/alhang/csv-files/master/gemini_BTCUSD_day.csv
- Number of observations: 2315
- Description: Daily Bitcoin data will be used to see crypto trends between 2019-2021. The data will indicate changes of the respective stock prices through the year


- Dataset Name: Covid data
- Link to the dataset:https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
- Number of observations: 782
- Description: The Covid data will provide data about covid cases in the US between January 2020 until now.


- Dataset Name: NASDAQ Historical data
- Link to the dataset: https://raw.githubusercontent.com/alhang/csv-files/master/NASDAQ_Historical.csv
- Number of observations: 1259
- Description: Stock market data from March 2019 to December 2021 will indicate stock prices through the year and their major changes. This dataset will be used to compare with the cryptocurrency prices, as stock prices are a good indicator of the economy and how it is affected during covid. 



# Setup

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest, t

In [2]:
eth_df_raw = pd.read_csv('https://raw.githubusercontent.com/alhang/csv-files/master/gemini_ETHUSD_day.csv', skiprows=1)
btc_df_raw = pd.read_csv('https://raw.githubusercontent.com/alhang/csv-files/master/gemini_BTCUSD_day.csv', skiprows=1)
covid_df_raw = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv')
nasdaq_df_raw = pd.read_csv('https://raw.githubusercontent.com/alhang/csv-files/master/NASDAQ_Historical.csv')

# Data Cleaning

Let's take a look at the raw Etheruem dataframe and see what needs cleaning.

In [3]:
eth_df_raw

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1644206400000,2022-02-07 04:00:00,ETHUSD,3072.90,3195.00,3052.59,3134.06,11611.670047
1,1644120000000,2022-02-06 04:00:00,ETHUSD,3011.36,3077.03,2947.69,3072.90,9226.545834
2,1644033600000,2022-02-05 04:00:00,ETHUSD,3000.46,3062.85,2964.23,3011.36,7247.098604
3,1643947200000,2022-02-04 04:00:00,ETHUSD,2685.66,3018.68,2678.05,3000.46,30992.365110
4,1643860800000,2022-02-03 04:00:00,ETHUSD,2659.30,2700.00,2576.87,2685.66,11232.318486
...,...,...,...,...,...,...,...,...
2096,1463112000,2016-05-13 04:00:00,ETHUSD,10.40,11.59,10.20,10.50,1788.043079
2097,1463025600,2016-05-12 04:00:00,ETHUSD,10.45,12.00,9.92,10.40,2061.012106
2098,1462939200,2016-05-11 04:00:00,ETHUSD,9.74,10.47,9.69,10.45,3079.531401
2099,1462852800,2016-05-10 04:00:00,ETHUSD,9.36,9.95,9.36,9.74,764.628290


We clean the Ethereum data to only have the date, closing price, and volume from January 21, 2020 to Feburary 7, 2022. We also need to reverse the index order and store the result in **eth_df**.

In [4]:
eth_df = eth_df_raw.iloc[:749]
eth_df = eth_df.sort_values(by=['Date'], ascending=True)
eth_df = eth_df.reset_index()
eth_df = eth_df.drop(columns=['Unix Timestamp','Symbol','Open','High','Low','index'])
eth_df

,Date,Close,Volume
0,2020-01-21 04:00:00,168.40,7328.818633
1,2020-01-22 04:00:00,164.86,17002.813084
2,2020-01-23 04:00:00,159.13,778.257216
3,2020-01-24 04:00:00,158.89,7895.695345
4,2020-01-25 04:00:00,161.18,3947.211172
...,...,...,...
744,2022-02-03 04:00:00,2685.66,11232.318486
745,2022-02-04 04:00:00,3000.46,30992.365110
746,2022-02-05 04:00:00,3011.36,7247.098604
747,2022-02-06 04:00:00,3072.90,9226.545834


Let's take a look at the raw Bitcoin dataframe and see what needs cleaning.

In [5]:
btc_df_raw

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1644206400000,2022-02-07 04:00:00,BTCUSD,42863.79,44543.78,42238.63,43857.45,1261.930492
1,1644120000000,2022-02-06 04:00:00,BTCUSD,41485.00,42980.00,41093.90,42863.79,750.096337
2,1644033600000,2022-02-05 04:00:00,BTCUSD,41487.51,41942.68,40971.46,41485.00,507.977777
3,1643947200000,2022-02-04 04:00:00,BTCUSD,37330.90,41996.48,37270.19,41487.51,2401.067754
4,1643860800000,2022-02-03 04:00:00,BTCUSD,36839.21,37454.49,36272.06,37330.90,1133.191332
...,...,...,...,...,...,...,...,...
2310,1444622400,2015-10-12 04:00:00,BTCUSD,248.98,248.98,245.75,245.75,71.047743
2311,1444536000,2015-10-11 04:00:00,BTCUSD,246.30,249.50,245.96,248.98,22.747091
2312,1444449600,2015-10-10 04:00:00,BTCUSD,245.51,246.30,244.60,246.30,30.870549
2313,1444363200,2015-10-09 04:00:00,BTCUSD,243.60,249.97,243.60,245.51,61.587068


We clean the Bitcoin data to only have the date, closing price, and volume from January 21, 2020 to Feburary 7, 2022. We also need to reverse the index order and store the result in **btc_df**.

In [6]:
btc_df = btc_df_raw.iloc[:749]
btc_df = btc_df.sort_values(by=['Date'], ascending=True)
btc_df = btc_df.reset_index()
btc_df = btc_df.drop(columns=['Unix Timestamp','Symbol','Open','High','Low','index'])
btc_df

,Date,Close,Volume
0,2020-01-21 04:00:00,8697.93,876.974281
1,2020-01-22 04:00:00,8559.71,585.652153
2,2020-01-23 04:00:00,8310.20,1649.808428
3,2020-01-24 04:00:00,8286.66,852.842705
4,2020-01-25 04:00:00,8346.01,308.447364
...,...,...,...
744,2022-02-03 04:00:00,37330.90,1133.191332
745,2022-02-04 04:00:00,41487.51,2401.067754
746,2022-02-05 04:00:00,41485.00,507.977777
747,2022-02-06 04:00:00,42863.79,750.096337


Let's take a look at the raw Covid cases dataframe and see what needs cleaning.

In [7]:
covid_df_raw

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0
...,...,...,...
777,2022-03-08,79270608,960690
778,2022-03-09,79318304,962503
779,2022-03-10,79356489,964062
780,2022-03-11,79400847,965746


We clean the covid dataset to only have the date and cases from January 21, 2020 to Feburary 7, 2022 and reverse the order of the indices and store the result in **covid_df**.

In [8]:
covid_df = covid_df_raw.drop(columns=['deaths'])
covid_df = covid_df.iloc[:749]
covid_df

,date,cases
0,2020-01-21,1
1,2020-01-22,1
2,2020-01-23,1
3,2020-01-24,2
4,2020-01-25,3
...,...,...
744,2022-02-03,75924121
745,2022-02-04,76236325
746,2022-02-05,76359918
747,2022-02-06,76418395


We also want a dataframe that has the number of new cases per day instead of total cases.

We initialize a new dataframe **newCases_df** that has the dates that we want with a column named "New Cases" and make all values in the column NaN.

In [16]:
newCases_df = covid_df.drop(columns=['cases'])
newCases_df['New Cases'] = np.nan
newCases_df

,date,New Cases
0,2020-01-21,NaN
1,2020-01-22,NaN
2,2020-01-23,NaN
3,2020-01-24,NaN
4,2020-01-25,NaN
...,...,...
744,2022-02-03,NaN
745,2022-02-04,NaN
746,2022-02-05,NaN
747,2022-02-06,NaN


Next, we iterate through the dates of **covid_df** and take the total number of cases of the current date minus the total number of cases of the previous date to find the change in cases from one day to the next. And we make this value the number of new cases on the current date in **newCases_df**.

In [17]:
i = 0
while i < covid_df.shape[0]:
    if i == 0:
        newCases_df.at[i,'New Cases'] = covid_df.iloc[i]['cases']
    else:
        newCases_df.at[i,'New Cases'] =  covid_df.iloc[i]['cases'] - covid_df.iloc[i-1]['cases']
    i += 1
newCases_df

,date,New Cases
0,2020-01-21,1.0
1,2020-01-22,0.0
2,2020-01-23,0.0
3,2020-01-24,1.0
4,2020-01-25,1.0
...,...,...
744,2022-02-03,312242.0
745,2022-02-04,312204.0
746,2022-02-05,123593.0
747,2022-02-06,58477.0


We need to standardize the dates for **btc_df** and **eth_df** to not include the time. This process will make analysis of the data easier.

In [ ]:
def standardize_date(string):
    string = string.strip()
    
    string = string.replace(' 04:00:00','')
    
    if '/' in string:
        string = string[6:10]+'-'+string[0:2]+'-'+string[3:5]
    
    return string

We apply the standarize_date function to the **btc_df**.

In [ ]:
btc_df['Date'] = btc_df['Date'].apply(standardize_date)
btc_df

Next, we apply the standarize_date function again to the **eth_df**.

In [ ]:
eth_df['Date'] = eth_df['Date'].apply(standardize_date)
eth_df

Now, the timestamps are removed.

Let's take a look at the raw Nasdaq dataframe and see what needs cleaning.

In [ ]:
nasdaq_df_raw

We clean the Nasdaq data to only include the date and the closing price from January 21, 2021 to now. We also need to reverse the index order and change the date from MM/DD/YY format to YY/MM/DD format and store the result in **nasdaq_df**.

In [ ]:
nasdaq_df_raw['Date'] = nasdaq_df_raw['Date'].apply(standardize_date)
nasdaq_df = nasdaq_df_raw.iloc[:521]
nasdaq_df = nasdaq_df.sort_values(by=['Date'], ascending=True)
nasdaq_df = nasdaq_df.reset_index()
nasdaq_df = nasdaq_df.drop(columns=['Volume','Open','High','Low','index'])
nasdaq_df = nasdaq_df.iloc[:518]
nasdaq_df

However, there is an issue here. Even though the start and end dates of the dataframe are the same of the ones for **btc_df**, **eth_df**, and **covid_df**, there are less observations. This is due to the stock markets being closed on weekends and holidays. So, the next step of our data cleaning is to fill in the missing dates with the close prices of the last day the market were open. For example, if the markets were closed for the weekend, we'd use Firday's close price to fill in the missing dates. This process will make analysis more accurate as the dates will line up with the other dataframes.

Firstly, we take the usable data from **nasdaq_df** and store it in a temporary variable, **nasdaq_df_old**.

In [ ]:
nasdaq_df_old = nasdaq_df

Then, we replace the dates in **nasdaq_df** with all the continuous dates from 2020-01-21 to 2022-02-07. This will take care of the missing dates. And then we turn the Close price to NaN values. We will fill these NaN values with the Close prices of temporary dataframe in the next step.

In [ ]:
nasdaq_df = btc_df.drop(columns=['Close'])
nasdaq_df['Close'] = np.nan
nasdaq_df

This loop here will fill in the the missing Close prices with the Close prices of the last day the market was open. It does this by comparing the dates of the temporary dataframe **nasdaq_df_old** and the dates of the dataframe that will be used, **nasdaq_df**. If **nasdaq_df** has a date that **nasdaq_df_old** does not have, it fills in the Close price of the last day the market was open. If the dates are the same, then it just takes the Close price of that date from **nasdaq_df_old** to **nasdaq_df**.

In [ ]:
i = 0
j = 0
while i < nasdaq_df.shape[0]:
    if nasdaq_df_old.iloc[j]['Date'] != nasdaq_df.iloc[i]['Date']:
        nasdaq_df.at[i,'Close'] = nasdaq_df_old.iloc[j-1]['Close/Last']
    else:
        nasdaq_df.at[i,'Close'] = nasdaq_df_old.iloc[j]['Close/Last']
        j+=1
    i+=1
nasdaq_df

We can see now that nasdaq_df has the same number of observations as the other dataframes. We can also see that for dates 2022-02-05 and 2022-02-06, which did not exist in **nasdaq_df_old**, that they have the Close price of the last day the market was open which was on 2022-02-04.

To check for missing values in all datasets, we run:

In [ ]:
eth_df.isnull().values.any()

In [ ]:
btc_df.isnull().values.any()

In [ ]:
covid_df.isnull().values.any()

In [ ]:
nasdaq_df.isnull().values.any()

Now all our datasets are clean and correctly formatted for analysis.

# Data Analysis & Results (EDA)

### Linear Regression Modeling on the Nasdaq and Bitcoin Dataframes

In [ ]:
x1 = nasdaq_df['Close']
y1 = btc_df['Close']
res1 = stats.linregress(x1, y1)
print(f"R-squared: {res1.rvalue**2:.6f}")
print(f"r-value: {res1.rvalue:.6f}")
plt.plot(x1, y1, 'o', label='original data')
plt.plot(x1, res1.intercept + res1.slope*x1, 'r', label='fitted line')
plt.title('Nasdaq and Bitcoin')
plt.legend()
plt.show()

## Description

### What distributions do your variables take?

Our data (specifically, NASDAQ and Bitcoin correlation here) is taken for linear correlation, and does not show normal distribution. 


### Are there any outliers?

From the data it shows that there are outliers when the nasdaq indicator reached around 12500 where bitcoin price starts swinging heavily. The most extreme outlier is at the point where the nasdaq indicator reaches 14000, where the Bitcoin price varies from 30000 to 70000. 


### Relationship between Variables?

The r-value is 0.877282, which indicates that there is a positive correlation between Nasdaq and Bitcoin, and such correlation is strong since ~0.877 is close to 1.0. 

### Linear Regression Modeling on the Covid Cases and Bitcoin Dataframes

In [ ]:
x2 = covid_df['cases']
y2 = btc_df['Close']
res2 = stats.linregress(x2, y2)
print(f"R-squared: {res2.rvalue**2:.6f}")
print(f"r-value: {res2.rvalue:.6f}")
plt.plot(x2, y2, 'o', label='original data')
plt.plot(x2, res2.intercept + res2.slope*x2, 'r', label='fitted line')
plt.title('Covid Cases and Bitcoin')
plt.legend()
plt.show()

## Description

### What distributions do your variables take?

Our data (specifically, COVID Cases and Bitcoin correlation here) is taken for linear correlation, and does not show normal distribution. 


### Are there any outliers?

From the data it shows that there are outliers when the covid cases reached around 3e^7 and 4.6e^7, the Bitcoin price start to swing heavily, ranging from 30000 to 65000. The maximum price is even higher at 4.6e^7. These two points can be counted as outliers for the model. 


### Relationship between Variables?

The r-value is 0.842249, which indicates that there is a positive correlation between COVID cases and Bitcoin, and such correlation is strong since ~0.842 is close to 1.0. 

 
 

### Linear Regression Modeling on the Covid Cases and Nasdaq Dataframes

In [ ]:
x3 = covid_df['cases']
y3 = nasdaq_df['Close']
res3 = stats.linregress(x3, y3)
print(f"R-squared: {res3.rvalue**2:.6f}")
print(f"r-value: {res3.rvalue:.6f}")
plt.plot(x3, y3, 'o', label='original data')
plt.plot(x3, res3.intercept + res3.slope*x3, 'r', label='fitted line')
plt.title('Covid Cases and Nasdaq')
plt.legend()
plt.show()

## Description

### What distributions do your variables take?

Our data (specifically, COVID Cases and Nasdaq correlation here) is taken for linear correlation, and does not show normal distribution. 


### Are there any outliers?

From the data it shows that there are outliers when the covid cases reached around 5.5e^7, the Nasdaq indicator starts to lower from 16000 to 14000. 


### Relationship between Variables?

The r-value is 0.886718, which indicates that there is a positive correlation between COVID cases and Bitcoin, and such correlation is strong since ~0.88 is close to 1.0. 

# Ethics & Privacy

Concerning the topic of privacy, our group should have very little to worry about. We are looking at data at a world-wide scale instead of using surveys, which have a potential to leak private information. All of our data will be sourced from aggragate data that will contain not personal information. Even though we have no privacy concerns, we're faced with other ethical issues. People can potentially misinterpret our analysis and only use covid cases as a predictor for the prices of cyrptocurrency, when in reality, the prices of crypto are influenced by a multitude and variables. Covid cases should not be the sole variable that is considered when predicting cyrptocurrency prices. Additionally there is an inherent bias in our anaylsis where we are observing the largest cryptocurrencies, Bitcoin and Ethereum, while the correlation may be different for smaller coins. The results from our project may influence people's decisions regarding the buying or selling of smaller coins, where the correlation we found for Bitcoin and Ethereum may not hold true. 

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

- Marco Paredes
    - TODO
- Alan Hang
    - Data Cleaning/ General Formatting/ Ethics
- Zehong Li
    - TODO
- Danyal Iqbal
    - TODO